## Ziehe alle Sätze aus einer Webseite, bereinige Sie und speichere als CSV


Prompt:

Schreibe einen Python Code, der folgende Funktionen erfüllt:
- Extrahieren aus einer Webseite alle Sätze
- Extrahiere auch aus allen verlinkten Seiten aus der Hauptseite innerhalb der Domäne
- max 10 links probieren
- die Sätze sollen forbereitet werden zur embedding verarbeitung mit openai
- Textblöcke säubern und korrigieren
- Entfernen von Zeilenumbrüchen und Leerzeichen am Anfang und Ende
- nur sätze nehmen die mindestens 10 Wörter hat
- es soll als csv gespeichert werden
- der code soll produktionsreif sein
- der code soll eine hohe qualität aufweisen

In [1]:
import csv
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

MAX_LINKS = 10

def prepare_text(sentence):
    sentence = sentence.strip()  # Leerzeichen am Anfang und Ende entfernen
    sentence = sentence.replace('\n', ' ')  # Zeilenumbrüche entfernen
    return sentence

def extract_sentences(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extrahiere Sätze aus der aktuellen Seite
        sentences = []
        for paragraph in soup.find_all('p'):
            raw_text = paragraph.get_text()
            sentences.extend(raw_text.split('. '))

        # Extrahiere Links und verfolge sie innerhalb der Domäne
        domain = urlparse(url).netloc
        links = set()
        for link in soup.find_all('a', href=True):
            if len(links) >= MAX_LINKS:
                break
            href = link['href']
            if href.startswith('/') or domain in href:
                absolute_url = urljoin(url, href)
                links.add(absolute_url)

        # Extrahiere Sätze aus den verlinkten Seiten
        for link in links:
            response = requests.get(link)
            linked_soup = BeautifulSoup(response.content, 'html.parser')
            for paragraph in linked_soup.find_all('p'):
                raw_text = paragraph.get_text()
                sentences.extend(raw_text.split('. '))

        # Sätze vorbereiten und filtern
        sentences = [prepare_text(sentence) for sentence in sentences if len(sentence.split()) >= 10]

        return sentences
    except:
        return []

def save_to_csv(sentences, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Sentence'])
        writer.writerows([[sentence] for sentence in sentences])





In [2]:
url = 'https://www.example.com'  # Setze hier die URL deiner gewünschten Webseite ein
sentences = extract_sentences(url)
save_to_csv(sentences, 'sentences.csv')